4-1-4 Autoencoder trained on GHZ states

In [ ]:
pip install cirq

In [ ]:
pip install pyswarms

In [ ]:
import numpy as np
import scipy as sc
#import random
import cirq
import pyswarms as ps

In [ ]:
#Generates SU(n) matrix given n^2 - 1 linearly independant params

def SUn(params, n):
    params = np.append(params, 0)
    params = np.mod(params, 2*np.pi)
    params = np.array(params, dtype='complex')
    params = params.reshape(n,n)
    z = np.zeros(n**4, dtype='complex')
    y = np.zeros(n**4, dtype='complex')
    z = z.reshape(n,n,n,n)
    y = y.reshape(n,n,n,n)
    
    for i in range(n):
        for j in range(n):
            if(i==j):
                zr = np.zeros(n**2)
                zr = zr.reshape(n,n)
                z[i][i] = zr
                y[i][i] = zr
            else:    
                z[i][j][i][i] = 1
                z[i][j][j][j] = -1
                y[i][j][i][j] = -1j
                y[i][j][j][i] = 1j
    
    for i in range(n):
        for j in range(i):
            params[j][i] = params[j][i]/4
            params[i][j] = params[i][j]/2
    U = np.zeros(n**2, dtype='complex')
    U = U.reshape(n,n)
    d=n
    
    for m in range(d-1):
        for n1 in range(m+1, d):
            U += 1j*params[n1][m]*z[m][n1] + 1j*params[m][n1]*y[m][n1]
    
    for l in range(d-1):
        U += 1j*params[l][l]*z[l][d-1]
    
    U = sc.linalg.expm(U)
    return U

In [ ]:
dtarget = np.zeros(256, dtype='complex')
dtarget = dtarget.reshape(16,16)
dtarget[0][0] = 0.5
dtarget[15][15] = 0.5
dtarget[0][15] = 0.5
dtarget[15][0] = 0.5

noise = 0.0   #Enter noise level here (0.0 for no noise)
def forward_prop(params):
    q = cirq.GridQubit.rect(9,1)
    u1_params = params[0:1023]
    u2_params = params[1023:2046]
    cir = cirq.Circuit()
    cir.append(cirq.BitFlipChannel(noise).on_each(q[0],q[1],q[2],q[3]))
    
    cir += cirq.H(q[0])
    cir += cirq.CNOT(q[0], q[1])
    cir += cirq.CNOT(q[1], q[2])
    cir += cirq.CNOT(q[2], q[3])
    cir += cirq.MatrixGate(SUn(u1_params, 32)).on(q[0],q[1],q[2],q[3],q[4])
    
    cir2 = cirq.Circuit()
    cir2 += cirq.MatrixGate(SUn(u2_params, 32)).on(q[4],q[5],q[6],q[7],q[8])
    cir2+=cirq.MeasurementGate(1).on(q[4])
    
    d1 = cirq.partial_trace(cirq.final_density_matrix(cir).reshape(2,2,2,2,2,2,2,2,2,2), keep_indices=[4]).reshape(2,2)
    rho_zero = np.zeros(256, dtype='complex')
    rho_zero = rho_zero.reshape(16,16)
    rho_zero[0][0] = 1
    
    dfinal = cirq.partial_trace(cirq.final_density_matrix(cir2, initial_state=np.kron(d1,rho_zero)).reshape(2,2,2,2,2,2,2,2,2,2), keep_indices=[1,2,3,4]).reshape(16,16)
    cost = sc.linalg.norm(dfinal-dtarget)
    
    return cost

In [ ]:
#For testing only..
params = np.load('..weights/GHZ_noiseless(4-1-4).npy')
print(forward_prop(params))

In [ ]:
#timing the function for optimization
from timeit import default_timer as timer
params = [random.uniform(0,2*np.pi) for _ in range(2298)]
st = timer()
print(forward_prop(stats[1]))
en = timer()
print(en-st)

In [ ]:
#@jit(target="cpu")
def f(x):
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [ ]:
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}


optimizer = ps.single.GlobalBestPSO(n_particles=250, dimensions=2046, options=options)

# Perform optimization

stats = optimizer.optimize(f, iters=1300, n_processes=4)


In [ ]:
#Save the trained weights for analysis 
np.save('ghz.npy', stats[1])